In [1]:
#%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
import os

In [3]:
# from fastai.column_data import *
# from fastai.conv_learner import *
# from fastai.dataset import *
from fastai.torch_imports import save_model, load_model

In [4]:
from models import *
from utils import *

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [5]:
# Experiment Code

In [6]:
EXP_CODE = "SOTA"

In [7]:
! ls -f KITTI/ | grep .csv

validation.csv
validation_192_640.csv
training.csv
validation_192_640_pre.csv
training_192_640_pre.csv
training_192_640.csv


In [8]:
train, val = pd.read_csv('./KITTI/training_192_640_pre.csv'), pd.read_csv('./KITTI/validation_192_640_pre.csv')
train, val = shuffle(train), shuffle(val);

In [9]:
train.head()

,t0,t1,t2,fx,fy,cx,cy
0,2011_09_26/2011_09_26_drive_0036_sync/image_02...,2011_09_26/2011_09_26_drive_0036_sync/image_02...,2011_09_26/2011_09_26_drive_0036_sync/image_02...,371.806866,369.427302,314.104631,88.501248
1,2011_09_30/2011_09_30_drive_0033_sync/image_03...,2011_09_30/2011_09_30_drive_0033_sync/image_03...,2011_09_30/2011_09_30_drive_0033_sync/image_03...,364.362615,362.030694,310.151266,93.752525
2,2011_09_26/2011_09_26_drive_0015_sync/image_03...,2011_09_26/2011_09_26_drive_0015_sync/image_03...,2011_09_26/2011_09_26_drive_0015_sync/image_03...,371.806866,369.427302,314.104631,88.501248
3,2011_10_03/2011_10_03_drive_0027_sync/image_02...,2011_10_03/2011_10_03_drive_0027_sync/image_02...,2011_10_03/2011_10_03_drive_0027_sync/image_02...,370.424992,368.054272,312.885179,94.830438
4,2011_09_26/2011_09_26_drive_0032_sync/image_02...,2011_09_26/2011_09_26_drive_0032_sync/image_02...,2011_09_26/2011_09_26_drive_0032_sync/image_02...,371.806866,369.427302,314.104631,88.501248


In [10]:
val.head()

,t0,t1,t2,fx,fy,cx,cy
0,2011_10_03/2011_10_03_drive_0034_sync/image_03...,2011_10_03/2011_10_03_drive_0034_sync/image_03...,2011_10_03/2011_10_03_drive_0034_sync/image_03...,370.424992,368.054272,312.885179,94.830438
1,2011_09_26/2011_09_26_drive_0087_sync/image_02...,2011_09_26/2011_09_26_drive_0087_sync/image_02...,2011_09_26/2011_09_26_drive_0087_sync/image_02...,371.806866,369.427302,314.104631,88.501248
2,2011_09_26/2011_09_26_drive_0022_sync/image_02...,2011_09_26/2011_09_26_drive_0022_sync/image_02...,2011_09_26/2011_09_26_drive_0022_sync/image_02...,371.806866,369.427302,314.104631,88.501248
3,2011_09_26/2011_09_26_drive_0022_sync/image_02...,2011_09_26/2011_09_26_drive_0022_sync/image_02...,2011_09_26/2011_09_26_drive_0022_sync/image_02...,371.806866,369.427302,314.104631,88.501248
4,2011_09_26/2011_09_26_drive_0059_sync/image_02...,2011_09_26/2011_09_26_drive_0059_sync/image_02...,2011_09_26/2011_09_26_drive_0059_sync/image_02...,371.806866,369.427302,314.104631,88.501248


In [11]:
trn0, trn1, trn2, trn_camera = train.t0, train.t1,train.t2, train[['fx', 'fy', 'cx', 'cy']]
val0, val1, val2, val_camera  = val.t0, val.t1, val.t2, val[['fx', 'fy', 'cx', 'cy']]

In [12]:
trn_camera.head()

,fx,fy,cx,cy
0,371.806866,369.427302,314.104631,88.501248
1,364.362615,362.030694,310.151266,93.752525
2,371.806866,369.427302,314.104631,88.501248
3,370.424992,368.054272,312.885179,94.830438
4,371.806866,369.427302,314.104631,88.501248


In [13]:
arch = f
bs = 8
PATH = 'Fastai_TRN'
DPATH = "KITTI/"
verbose = 100

In [14]:
f

<function torchvision.models.resnet.resnet18(pretrained=False, **kwargs)>

In [15]:
trn0, trn1, trn2, trn_camera = train.t0, train.t1,train.t2, train[['fx', 'fy', 'cx', 'cy']]
val0, val1, val2, val_camera  = val.t0, val.t1, val.t2, val[['fx', 'fy', 'cx', 'cy']]

In [16]:
stats = imagenet_stats
tfm_norm = Normalize(*stats, tfm_y=TfmType.NO) 
tfm_denorm = Denormalize(*stats)

In [17]:
trn_tfms = UnScaleTransforms(sz=None, tfms=[], normalizer=tfm_norm, denorm=tfm_denorm, tfm_y=TfmType.NO, sz_y=None)
val_tfms = UnScaleTransforms(sz=None, tfms=[], normalizer=tfm_norm, denorm=tfm_denorm, tfm_y=TfmType.NO, sz_y=None)

In [18]:
tfms = (trn_tfms, val_tfms)

MD1 = get_MD(trn0, val0, tfms, bs, DPATH, PATH)
MD2 = get_MD(trn1, val1, tfms, bs, DPATH, PATH)
MD3 = get_MD(trn2, val2, tfms, bs, DPATH, PATH)
MDcam = get_cam(trn_camera, val_camera, bs)

In [19]:
imgnet_mean, imgnet_std = torch.from_numpy(stats[0]).float(), torch.from_numpy(stats[1]).float()
imgnet_mean, imgnet_std = imgnet_mean.view(1,3,1,1), imgnet_std.view(1,3,1,1)
denorm = denormer(imgnet_mean, imgnet_std)

In [20]:
def train(m, opt, MD1, MD2, MD3, MDcam, denorm):
    collector = LossCollect(6)
    
    m.train()
    DL1, DL2, DL3, DLcam = iter(MD1.trn_dl), iter(MD2.trn_dl), iter(MD3.trn_dl), iter(MDcam.trn_dl)
    losses = []  
    
    #for i in range(2): # just for testing
    for i in range(len(MD1.trn_ds)//bs-len(losses)):
        t1 = time.time()
        opt.zero_grad()
        imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]])
        
        d1, d2, d3, trans, rotation, = m(imgs1, imgs2, imgs3)
        #pdb.set_trace()
        imgs1, imgs2, imgs3 = denorm(imgs1), denorm(imgs2), denorm(imgs3) 
        #pdb.set_trace()
        loss, details = l(d1, d2, d3, trans, rotation, imgs1, imgs2, imgs3, cam)
        loss.backward()
        opt.step()
        t2 = time.time()
        
        collector.collect(
            loss.data[0],
            details[0].data[0],
            details[1].data[0],
            details[2].data[0],
            details[3].data[0],
            t2-t1
            )
        
        # total appr smooth ssim l1
        if i%verbose == 0: print("{:.4f}, {:.4f}, {:.4E}, {:.4f}, {:.4f}, {:.2f}s/batch".format(*collector.show()))
    return collector.collections[0]  #return the total losses

In [26]:
def evaluate(m, MD1, MD2, MD3, MDcam, folder):
    
    DL1, DL2, DL3, DLcam = iter(MD1.val_dl), iter(MD2.val_dl), iter(MD3.val_dl), iter(MDcam.val_dl)
    m.eval()
    for i in range(6): 
        imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]], volatile=True)
        d1s, d2s, d3s, trans, rotation, = m(imgs1, imgs2, imgs3)
        #pdb.set_trace()
        imgs1, imgs2, imgs3 = denorm(imgs1, volatile=True), denorm(imgs2, volatile=True), denorm(imgs3, volatile=True) 
        #pdb.set_trace()
        d2 = d2s[0]
        # d2 = F.upsample(input=d2, scale_factor=2**di, mode='bilinear')
        cx12, cy12, dm12 = l.appr.offset.forward(trans=trans[:,0], rotation=rotation[:,0], inv_depth = d2, camera = cam)
        #pdb.set_trace()

        x12, ivm12 = l.appr.sampler.forward(imgs1, cx12, cy12)
        ivm12.volatile = True

        del dm12 
        del ivm12
        
        imgs2 = tonp(imgs2.permute(0,2,3,1))
        x12 = tonp(x12.permute(0,2,3,1))
        d2 = tonp(d2)
        
        for j in range(bs):
            save_res(imgs2[j], x12[j], d2[j][0], folder/"{}{}.png".format(i,j))

In [22]:
def fit(epoch, m, opt, MD1, MD2, MD3, MDcam):
    for i in range(epoch):
        print("--------------------epoch {} start:----------------------".format(i))
        losses = train(m, opt, MD1, MD2, MD3, MDcam)
        losses= [ str(loss) for loss in losses ]
        folder = Path("./tmp")
        folder.mkdir(exist_ok=True)
        file = folder / 'epoch{}.log'.format(i)
        with file.open('w') as f:
            f.write("\n".join(losses))
        save_model(m, str(folder / "epoch{}.M".format(i)))

In [23]:
expfolder = Path('./experiment/')/EXP_CODE
expfolder.mkdir(parents=True, exist_ok=True)

In [24]:
m = TriDepth(get_base(f, cut), 1, setting=[False, False, False]).cuda()
l = Loss(smooth_scale=0.001, appr_scale=0.85, warp_setting=[True, True, True, True], is_per_pixel_min=True, is_mask=False).cuda()
opt = optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr =1e-4)


# Run this if necessary
from fastai.torch_imports import save_model, load_model
path = expfolder/'epoch_2'/'model.M'
load_model(m, str(path))

In [27]:
start = 0 # should be 0 if normal 
epoch = 8
#     set_trainable(m.depth.rn, False)
for i in range(start,epoch):    
    thisfolder = expfolder/"epoch_{}".format(i+1)
    thisfolder.mkdir(exist_ok=True)

    print("Epoch {} start".format(i+1))
    print("------------------training model--------------------")
    losses = train(m, opt, MD1, MD2, MD3, MDcam, denorm)
    print("------------------saving model-------------------")
    save_model(m, thisfolder/"model.M")
    print("------------------writting log-------------------")
    logger = thisfolder/"log"
    losses = [ str(loss) for loss in losses]
    with logger.open('w') as f: f.write("\n".join(losses))
    print("---------------------predicting------------------")    
    predictfolder = thisfolder/'predicts'
    predictfolder.mkdir(exist_ok=True)
    evaluate(m, MD1, MD2, MD3, MDcam, predictfolder)

------------------training model-----------------
0.1546, 0.1546, 3.1577E-05, 0.1546, 0.1546, 13.12s/batch
0.1278, 0.1277, 1.9282E-05, 0.1277, 0.1277, 1.23s/batch
0.1291, 0.1291, 2.3663E-05, 0.1291, 0.1291, 1.20s/batch
0.1154, 0.1154, 2.1620E-05, 0.1154, 0.1154, 1.25s/batch
0.1065, 0.1065, 1.6381E-05, 0.1065, 0.1065, 1.20s/batch
0.1021, 0.1021, 1.3580E-05, 0.1021, 0.1021, 1.25s/batch
0.0936, 0.0936, 1.4679E-05, 0.0936, 0.0936, 1.20s/batch
0.0932, 0.0932, 1.2430E-05, 0.0932, 0.0932, 1.20s/batch
0.0973, 0.0973, 1.2836E-05, 0.0973, 0.0973, 2.62s/batch
0.1019, 0.1019, 1.3509E-05, 0.1019, 0.1019, 1.19s/batch
0.0966, 0.0965, 1.2733E-05, 0.0965, 0.0965, 1.17s/batch
0.0974, 0.0974, 1.3678E-05, 0.0974, 0.0974, 1.21s/batch
0.0954, 0.0954, 1.2936E-05, 0.0954, 0.0954, 1.23s/batch
0.1064, 0.1064, 1.2790E-05, 0.1064, 0.1064, 1.51s/batch
0.0993, 0.0993, 1.2100E-05, 0.0993, 0.0993, 1.20s/batch
0.0898, 0.0898, 1.2705E-05, 0.0898, 0.0898, 1.21s/batch
0.0893, 0.0893, 1.2086E-05, 0.0893, 0.0893, 2.58s/bat

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


------------------training model-----------------
0.0715, 0.0715, 1.1095E-05, 0.0715, 0.0715, 7.87s/batch
0.0882, 0.0882, 1.0245E-05, 0.0882, 0.0882, 1.27s/batch
0.0877, 0.0877, 1.0490E-05, 0.0877, 0.0877, 1.24s/batch
0.0791, 0.0791, 1.1186E-05, 0.0791, 0.0791, 1.30s/batch
0.0764, 0.0764, 1.0601E-05, 0.0764, 0.0764, 1.27s/batch
0.0753, 0.0753, 1.0244E-05, 0.0753, 0.0753, 1.25s/batch
0.0722, 0.0722, 1.0628E-05, 0.0722, 0.0722, 1.25s/batch
0.0703, 0.0703, 1.0053E-05, 0.0703, 0.0703, 1.22s/batch
0.0724, 0.0724, 1.0586E-05, 0.0724, 0.0724, 2.44s/batch
0.0801, 0.0801, 1.0584E-05, 0.0801, 0.0801, 1.18s/batch
0.0735, 0.0735, 1.0314E-05, 0.0735, 0.0735, 1.22s/batch
0.0744, 0.0744, 1.0416E-05, 0.0744, 0.0744, 1.37s/batch
0.0722, 0.0722, 1.0836E-05, 0.0722, 0.0722, 1.26s/batch
0.0810, 0.0810, 1.0697E-05, 0.0810, 0.0810, 1.18s/batch
0.0790, 0.0790, 1.0311E-05, 0.0790, 0.0790, 1.21s/batch
0.0722, 0.0722, 1.0212E-05, 0.0722, 0.0722, 1.21s/batch
0.0741, 0.0741, 1.0505E-05, 0.0741, 0.0741, 2.36s/batc

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


------------------training model-----------------
0.0625, 0.0624, 1.0944E-05, 0.0624, 0.0624, 6.69s/batch
0.0782, 0.0782, 1.0012E-05, 0.0782, 0.0782, 1.22s/batch
0.0769, 0.0769, 9.7631E-06, 0.0769, 0.0769, 1.20s/batch
0.0707, 0.0707, 1.0698E-05, 0.0707, 0.0707, 1.23s/batch
0.0722, 0.0722, 1.0074E-05, 0.0722, 0.0722, 1.22s/batch
0.0710, 0.0710, 1.0177E-05, 0.0710, 0.0710, 1.20s/batch
0.0679, 0.0679, 1.0087E-05, 0.0679, 0.0679, 1.21s/batch
0.0680, 0.0680, 9.8211E-06, 0.0680, 0.0680, 1.17s/batch
0.0686, 0.0686, 9.8564E-06, 0.0686, 0.0686, 2.30s/batch
0.0755, 0.0755, 9.8660E-06, 0.0755, 0.0755, 1.17s/batch
0.0697, 0.0697, 9.7603E-06, 0.0697, 0.0697, 1.22s/batch
0.0692, 0.0692, 1.0165E-05, 0.0692, 0.0692, 1.23s/batch
0.0669, 0.0668, 1.0562E-05, 0.0668, 0.0668, 1.21s/batch
0.0774, 0.0774, 1.0468E-05, 0.0774, 0.0774, 1.23s/batch
0.0754, 0.0754, 9.9157E-06, 0.0754, 0.0754, 1.23s/batch
0.0686, 0.0686, 1.0285E-05, 0.0686, 0.0686, 1.22s/batch
0.0670, 0.0670, 9.9248E-06, 0.0670, 0.0670, 2.45s/batc

...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


------------------training model-----------------
0.0578, 0.0578, 1.0383E-05, 0.0578, 0.0578, 8.10s/batch
0.0673, 0.0673, 9.8324E-06, 0.0673, 0.0673, 1.20s/batch
0.0637, 0.0637, 9.8349E-06, 0.0637, 0.0637, 1.20s/batch
0.0599, 0.0599, 1.0339E-05, 0.0599, 0.0599, 1.37s/batch
0.0593, 0.0592, 9.5873E-06, 0.0592, 0.0592, 1.61s/batch
0.0628, 0.0628, 9.8949E-06, 0.0628, 0.0628, 1.26s/batch
0.0576, 0.0576, 9.7911E-06, 0.0576, 0.0576, 1.24s/batch
0.0588, 0.0588, 9.5972E-06, 0.0588, 0.0588, 1.19s/batch
0.0585, 0.0585, 9.9502E-06, 0.0585, 0.0585, 2.36s/batch
0.0663, 0.0663, 9.9516E-06, 0.0663, 0.0663, 1.19s/batch
0.0618, 0.0618, 9.3152E-06, 0.0618, 0.0618, 1.20s/batch
0.0656, 0.0656, 9.6730E-06, 0.0656, 0.0656, 1.25s/batch
0.0629, 0.0629, 1.0265E-05, 0.0629, 0.0629, 1.19s/batch
0.0673, 0.0673, 9.8671E-06, 0.0673, 0.0673, 1.23s/batch
0.0631, 0.0631, 9.6756E-06, 0.0631, 0.0631, 1.22s/batch
0.0588, 0.0588, 9.6459E-06, 0.0588, 0.0588, 1.21s/batch
0.0601, 0.0601, 9.8438E-06, 0.0601, 0.0601, 2.69s/batc

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


------------------training model-----------------
0.0555, 0.0555, 1.0166E-05, 0.0555, 0.0555, 7.15s/batch
0.0623, 0.0623, 1.0053E-05, 0.0623, 0.0623, 1.22s/batch
0.0621, 0.0620, 9.7301E-06, 0.0620, 0.0620, 1.20s/batch
0.0580, 0.0580, 1.0430E-05, 0.0580, 0.0580, 1.23s/batch
0.0572, 0.0572, 9.8009E-06, 0.0572, 0.0572, 1.24s/batch
0.0592, 0.0591, 1.0284E-05, 0.0591, 0.0591, 1.23s/batch
0.0572, 0.0572, 9.6660E-06, 0.0572, 0.0572, 1.32s/batch
0.0588, 0.0588, 9.7068E-06, 0.0588, 0.0588, 1.19s/batch
0.0578, 0.0578, 9.9071E-06, 0.0578, 0.0578, 2.39s/batch
0.0647, 0.0647, 9.7802E-06, 0.0647, 0.0647, 1.23s/batch
0.0610, 0.0610, 9.3341E-06, 0.0610, 0.0610, 1.19s/batch
0.0592, 0.0591, 9.5440E-06, 0.0591, 0.0591, 1.22s/batch
0.0572, 0.0572, 1.0099E-05, 0.0572, 0.0572, 1.20s/batch
0.0651, 0.0650, 1.0242E-05, 0.0650, 0.0650, 1.22s/batch
0.0602, 0.0602, 9.5222E-06, 0.0602, 0.0602, 1.24s/batch
0.0594, 0.0594, 9.6107E-06, 0.0594, 0.0594, 1.37s/batch
0.0567, 0.0567, 9.8206E-06, 0.0567, 0.0567, 2.27s/batc

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [ ]:
DL1, DL2, DL3, DLcam = iter(MD1.val_dl), iter(MD2.val_dl), iter(MD3.val_dl), iter(MDcam.val_dl)
m.eval()


In [ ]:
imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]], volatile=True)
d1s, d2s, d3s, trans, rotation, = m(imgs1, imgs2, imgs3)
        

In [ ]:
d2s[0].min(), d2s[0].max()